# Day 4: Passport Processing

Reference: https://adventofcode.com/2020/day/4

## Part 1

You arrive at the airport only to realize that you grabbed your North Pole Credentials instead of your passport. While these documents are extremely similar, North Pole Credentials aren't issued by a country and therefore aren't actually valid documentation for travel in most of the world.

It seems like you're not the only one having problems, though; a very long line has formed for the automatic passport scanners, and the delay could upset your travel itinerary.

Due to some questionable network security, you realize you might be able to solve both of these problems at the same time.

The automatic passport scanners are slow because they're having trouble detecting which passports have all required fields. The expected fields are as follows:

    byr (Birth Year)
    iyr (Issue Year)
    eyr (Expiration Year)
    hgt (Height)
    hcl (Hair Color)
    ecl (Eye Color)
    pid (Passport ID)
    cid (Country ID)

Passport data is validated in batch files (your puzzle input). Each passport is represented as a sequence of key:value pairs separated by spaces or newlines. Passports are separated by blank lines.

Here is an example batch file containing four passports:
```
ecl:gry pid:860033327 eyr:2020 hcl:#fffffd
byr:1937 iyr:2017 cid:147 hgt:183cm

iyr:2013 ecl:amb cid:350 eyr:2023 pid:028048884
hcl:#cfa07d byr:1929

hcl:#ae17e1 iyr:2013
eyr:2024
ecl:brn pid:760753108 byr:1931
hgt:179cm

hcl:#cfa07d eyr:2025 pid:166559648
iyr:2011 ecl:brn hgt:59in
```
The first passport is valid - all eight fields are present. The second passport is invalid - it is missing hgt (the Height field).

The third passport is interesting; the only missing field is cid, so it looks like data from North Pole Credentials, not a passport at all! Surely, nobody would mind if you made the system temporarily ignore missing cid fields. Treat this "passport" as valid.

The fourth passport is missing two fields, cid and byr. Missing cid is fine, but missing any other field is not, so this passport is invalid.

According to the above rules, your improved system would report 2 valid passports.

Count the number of valid passports - those that have all required fields. Treat cid as optional. In your batch file, how many passports are valid?

In [165]:
import itertools
import re
import numpy as np
from IPython.display import Markdown
from IPython.core.debugger import set_trace as breakpt

In [161]:
# Read in the passport data and do 
# some preliminary parsing
rlist = list()
record = dict()
with open('pwdb_input.txt', 'r') as fid:
    for line in fid:
        line = line.strip()
        for field in line.split():
            k, v = field.split(':')
            record[k] = v
        if not line and record:
            rlist.append(record.copy())
            record = dict()

In [160]:
# Create a list of valid records for entries that
# contain all requisite fields
reqkeys = {'byr', 'iyr', 'eyr', 'hgt', 'hcl', 'ecl', 'pid'}
vlist = list()
for rec in rlist:
    if not(reqkeys - set(rec.keys())):
        # If an empty set, all reqkeys were present
        # and record was valid
        #print("{} Valid: {}".format(i, ', '.join(list(rec.keys()))))
        vlist.append(rec)

In [166]:
#Markdown("Found **{}** valid passport records".format(len(vlist)))

## Part 2

The line is moving more quickly now, but you overhear airport security talking about how passports with invalid data are getting through. Better add some data validation, quick!

You can continue to ignore the cid field, but each other field has strict rules about what values are valid for automatic validation:

    byr (Birth Year) - four digits; at least 1920 and at most 2002.
    iyr (Issue Year) - four digits; at least 2010 and at most 2020.
    eyr (Expiration Year) - four digits; at least 2020 and at most 2030.
    hgt (Height) - a number followed by either cm or in:
        If cm, the number must be at least 150 and at most 193.
        If in, the number must be at least 59 and at most 76.
    hcl (Hair Color) - a # followed by exactly six characters 0-9 or a-f.
    ecl (Eye Color) - exactly one of: amb blu brn gry grn hzl oth.
    pid (Passport ID) - a nine-digit number, including leading zeroes.
    cid (Country ID) - ignored, missing or not.

Your job is to count the passports where all required fields are both present and valid according to the above rules. Here are some example values:
```
byr valid:   2002
byr invalid: 2003

hgt valid:   60in
hgt valid:   190cm
hgt invalid: 190in
hgt invalid: 190

hcl valid:   #123abc
hcl invalid: #123abz
hcl invalid: 123abc

ecl valid:   brn
ecl invalid: wat

pid valid:   000000001
pid invalid: 0123456789
```
Here are some invalid passports:
```
eyr:1972 cid:100
hcl:#18171d ecl:amb hgt:170 pid:186cm iyr:2018 byr:1926

iyr:2019
hcl:#602927 eyr:1967 hgt:170cm
ecl:grn pid:012533040 byr:1946

hcl:dab227 iyr:2012
ecl:brn hgt:182cm pid:021572410 eyr:2020 byr:1992 cid:277

hgt:59cm ecl:zzz
eyr:2038 hcl:74454a iyr:2023
pid:3556412378 byr:2007
```
Here are some valid passports:
```
pid:087499704 hgt:74in ecl:grn iyr:2012 eyr:2030 byr:1980
hcl:#623a2f

eyr:2029 ecl:blu cid:129 byr:1989
iyr:2014 pid:896056539 hcl:#a97842 hgt:165cm

hcl:#888785
hgt:164cm byr:2001 iyr:2015 cid:88
pid:545766238 ecl:hzl
eyr:2022

iyr:2010 hgt:158cm hcl:#b6652a ecl:blu byr:1944 eyr:2021 pid:093154719
```
Count the number of valid passports - those that have all required fields and valid values. Continue to treat cid as optional. In your batch file, how many passports are valid?

In [168]:
# Set up some useful variables for validation
rht = re.compile('(\d+)(cm|in)')
rhcl = re.compile('#[0-9a-f]{6}')
eyecolors = {'amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth'}
rpid = re.compile('[0-9]{9}')

# Keep a list of validated fields for debugging purposes
lbyr, liyr, leyr, lhgt = [list()] * 4
lhgtnum, lhgtunits, lhcl = [list()] * 3
lecl, lpid = [list()] * 2

# Slog through all the records. If any field does
# not verify, skip it and move on. If all the fields
# verify by the end of the loop, log it as verified.
verified = 0
for rec in vlist:
    # Check byr
    byr = int(rec['byr'])
    if not(1920 <= byr and 2002 >= byr):
        continue
    # Check iyr
    iyr = int(rec['iyr'])
    if not(2010 <= iyr and 2020 >= iyr):
        continue
    # Check eyr
    eyr = int(rec['eyr'])
    if not(2020 <= eyr and 2030 >= eyr):
        continue
    # Check hgt
    htm = rht.match(rec['hgt'])
    if htm is None:
        # No match means invalid height spec
        continue
    ht = int(htm[1])
    units = htm[2]
    if 'cm' == units:
        if not(150 <= ht and 193 >= ht):
            continue
    elif 'in' == units:
        if not(59 <= ht and 76 >= ht):
            continue
    else:
        # Invalid height units
        continue
    # Check hcl
    hclm = rhcl.match(rec['hcl'])
    if hclm is None:
        # Invalid hair color
        continue
    # Check ecl
    if not(rec['ecl'] in eyecolors):
        continue
    # Check pid
    pidm = rpid.match(rec['pid'])
    if pidm is None:
        # Invalid passport id
        continue
    elif len(pidm.string) > 9:
        # Check that not more than 9 digits were matched
        continue
    # Still here? Information was verified
    #print("byr: {}, iyr: {}, eyr: {}, hgt: {}, hcl: {}, ecl: {}, pid: {}"
    #      .format(byr, iyr, eyr, rec['hgt'], rec['hcl'], rec['ecl'], rec['pid']))
    verified += 1
    lbyr.append(byr)
    liyr.append(iyr)
    leyr.append(eyr)
    lhgt.append(rec['hgt'])
    lhgtnum.append(ht)
    lhgtunits.append(units)
    lhcl.append(rec['hcl'])
    lecl.append(rec['ecl'])
    lpid.append(rec['pid'])

In [137]:
# For debugging, make a DataFrame of the validated records
#import pandas as pd
#df = pd.DataFrame({'byr': lbyr, 'iyr': liyr, 'eyr': leyr,
#                   'hgt': lhgt, 'htnum': lhgtnum, 'htun': lhgtunits,
#                   'hcl': lhcl, 'ecl': lecl, 'pid': lpid})

# Show the range of validated 'cm' heights
#ght = df.groupby('htun')
#ght.get_group('cm')['htnum'].agg([np.min, np.max])
# Show the range of validated 'in' heights
#ght.get_group('in')['htnum'].agg([np.min, np.max])

In [170]:
#Markdown("Verified **{}** passport records".format(verified))